In [1]:
%load_ext dotenv
%dotenv 
# UPSTAGE_API_KEY from https://console.upstage.ai/

In [6]:
# @title set API key
import os
import getpass
from pprint import pprint
import warnings
import matplotlib.pyplot as plt 
import matplotlib.font_manager as fm

warnings.filterwarnings("ignore")

from IPython import get_ipython

if "google.colab" in str(get_ipython()):
    # Running in Google Colab. Please set the UPSTAGE_API_KEY in the Colab Secrets
    from google.colab import userdata
    os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")
else:
    # Running locally. Please set the UPSTAGE_API_KEY in the .env file
    from dotenv import load_dotenv

    load_dotenv()

if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")


In [ ]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# 문서 로딩 및 텍스트 분할
def load_and_split_docs(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    return splits

# 문서 임베딩 및 FAISS 인덱싱
def embed_and_index_docs_with_faiss(splits, faiss_index_path="faiss_index"):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# 산업군 추출 함수
def extract_industry(company_name):
    industry_prompt = f"'{company_name}'의 산업군을 알려주세요."
    return chat(industry_prompt, []).strip()

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name):
    industry = extract_industry(company_name)
    combined_query = f"{company_name} {industry}"
    
    if db_retriever:
        search_results = db_retriever.get_relevant_documents(combined_query)
        return "\n".join([doc.page_content for doc in search_results])
    return "관련 정보를 찾을 수 없습니다."

# 기업 분석 생성 함수
def generate_company_analysis(company_name):
    history = []
    related_info = retrieve_related_info(company_name)

    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수 (기업 분석 포함)
def generate_trade_recommendation_with_analysis(company_name):
    history = []
    
    # 기업 분석과 관련 정보 결합
    analysis = generate_company_analysis(company_name)
    related_info = retrieve_related_info(company_name)

    recommendation_message = (
        f"다음은 '{company_name}'에 대한 기업 분석과 관련 정보입니다:\n\n{analysis}\n\n"
        f"이 모든 정보를 바탕으로 반드시 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요. "
        f"단, 추천 이유와 함께 고려해야 할 추가 사항도 설명해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 최종 매수/매도 결정 생성 함수
def generate_final_decision(analysis, recommendation):
    history = []
    
    final_decision_message = (
        f"다음은 기업 분석과 매수/매도 추천 내용입니다. "
        f"이 모든 정보를 종합하여 최종적으로 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요:\n\n"
        f"기업 분석:\n{analysis}\n\n매수/매도 추천:\n{recommendation}"
    )
    final_decision = chat(final_decision_message, history)
    
    # 최종 결정 추출
    if "매수" in final_decision:
        return "매수"
    elif "매도" in final_decision or "중립" in final_decision or "hold" in final_decision.lower():
        return "매도"
    else:
        return "추천 없음"

# 통합 함수: 기업 분석과 매수/매도 추천 및 최종 결론 출력
def generate_analysis_recommendation_decision(company_name):
    if not company_name:
        return "기업명을 입력해 주세요.", "", ""
    
    # 기업 분석 및 매수/매도 추천 생성
    analysis = generate_company_analysis(company_name)
    recommendation = generate_trade_recommendation_with_analysis(company_name)
    
    # 최종 매수/매도 결론 생성
    final_decision = generate_final_decision(analysis, recommendation)
    
    return analysis, recommendation, final_decision

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("기업명을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="종목명", placeholder="예: 삼성전자")

    # 기업명 아래에 버튼 추가
    submit_btn = gr.Button("분석 및 추천 받기")

    # 기업 분석과 매수/매도 추천 및 최종 결정 결과 표시를 위한 텍스트 박스 컴포넌트 추가
    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    # 버튼 클릭 이벤트 설정 및 연결
    submit_btn.click(
        generate_analysis_recommendation_decision,
        inputs=[company_name],
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

In [38]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
from langchain.schema import AIMessage, HumanMessage
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import sqlite3

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# 문서 로딩 및 텍스트 분할
def load_and_split_docs(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    return splits

# 문서 임베딩 및 FAISS 인덱싱
def embed_and_index_docs_with_faiss(splits, faiss_index_path="faiss_index"):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# 산업군 추출 함수
def extract_industry(company_name):
    industry_prompt = f"'{company_name}'의 산업군을 알려주세요."
    return chat(industry_prompt, []).strip()

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name):
    industry = extract_industry(company_name)
    combined_query = f"{company_name} {industry}"
    
    if db_retriever:
        search_results = db_retriever.get_relevant_documents(combined_query)
        return "\n".join([doc.page_content for doc in search_results])
    return "관련 정보를 찾을 수 없습니다."

# specific_info.db에서 기업 정보를 가져오는 함수
def get_company_info_from_db(company_name):
    conn = sqlite3.connect("specific_info.db")
    cursor = conn.cursor()
    
    # 회사명으로 정보 검색
    query = f"SELECT * FROM company_info WHERE 종목명 = ?"
    cursor.execute(query, (company_name,))
    result = cursor.fetchone()
    
    conn.close()
    
    if result:
        # 필요한 정보를 텍스트로 정리
        info_text = f"종목명: {result[1]}\n주당액면가액: {result[2]}\n당기순이익: {result[3]} 백만원\n" \
                    f"주당순이익: {result[4]} 원\n현금배당금총액: {result[5]} 백만원\n" \
                    f"현재가: {result[6]}\n시장 가치: {result[7]}"
        return info_text
    return "해당 기업의 정보를 찾을 수 없습니다."

# 기업 분석 함수 수정 (DB에서 정보 포함)
def generate_company_analysis_with_db(company_name):
    history = []
    
    # specific_info.db에서 기업 정보 가져오기
    db_info = get_company_info_from_db(company_name)
    if db_info == "해당 기업의 정보를 찾을 수 없습니다.":
        return db_info
    
    # 관련 정보 검색 및 분석
    related_info = retrieve_related_info(company_name)
    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{db_info}\n\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수 (기업 분석 포함)
def generate_trade_recommendation_with_analysis(company_name):
    history = []
    
    # 기업 분석과 관련 정보 결합
    analysis = generate_company_analysis(company_name)
    related_info = retrieve_related_info(company_name)

    recommendation_message = (
        f"다음은 '{company_name}'에 대한 기업 분석과 관련 정보입니다:\n\n{analysis}\n\n"
        f"이 모든 정보를 바탕으로 반드시 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요. "
        f"단, 추천 이유와 함께 고려해야 할 추가 사항도 설명해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# Gradio 통합 함수 수정
def generate_analysis_recommendation_decision_with_db(company_name):
    if not company_name:
        return "기업명을 입력해 주세요.", "", ""
    
    # DB 기반 기업 분석 및 매수/매도 추천 생성
    analysis = generate_company_analysis_with_db(company_name)
    recommendation = generate_trade_recommendation_with_analysis(company_name)
    
    # 최종 매수/매도 결론 생성
    final_decision = generate_final_decision(analysis, recommendation)
    
    return analysis, recommendation, final_decision

# 최종 매수/매도 결정 생성 함수
def generate_final_decision(analysis, recommendation):
    history = []
    
    final_decision_message = (
        f"다음은 기업 분석과 매수/매도 추천 내용입니다. "
        f"이 모든 정보를 종합하여 최종적으로 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요:\n\n"
        f"기업 분석:\n{analysis}\n\n매수/매도 추천:\n{recommendation}"
    )
    final_decision = chat(final_decision_message, history)
    
    # 최종 결정 추출
    if "매수" in final_decision:
        return "매수"
    elif "매도" in final_decision or "중립" in final_decision or "hold" in final_decision.lower():
        return "매도"
    else:
        return "추천 없음"

# 통합 함수: 기업 분석과 매수/매도 추천 및 최종 결론 출력
def generate_analysis_recommendation_decision(company_name):
    if not company_name:
        return "기업명을 입력해 주세요.", "", ""
    
    # 기업 분석 및 매수/매도 추천 생성
    analysis = generate_company_analysis(company_name)
    recommendation = generate_trade_recommendation_with_analysis(company_name)
    
    # 최종 매수/매도 결론 생성
    final_decision = generate_final_decision(analysis, recommendation)
    
    return analysis, recommendation, final_decision

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("기업명을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="종목명", placeholder="예: 삼성전자")

    # 기업명 아래에 버튼 추가
    submit_btn = gr.Button("분석 및 추천 받기")


    # 기업 분석과 매수/매도 추천 및 최종 결정 결과 표시를 위한 텍스트 박스 컴포넌트 추가
    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    # 버튼 클릭 이벤트 설정 및 연결
    # Gradio 인터페이스 수정
    submit_btn.click(
    generate_analysis_recommendation_decision_with_db,
    inputs=[company_name],
    outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

if __name__ == "__main__":
    file_path = "hana.pdf"
    splits = load_and_split_docs(file_path)
    db_retriever = embed_and_index_docs_with_faiss(splits, "faiss_index")
    demo.launch()

* Running on local URL:  http://127.0.0.1:7925

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/blocks.py", line 2018, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/blocks.py", line 1567, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/L

In [39]:
import os
import gradio as gr
import pandas as pd
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS  
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# 산업군 추출 함수
def extract_industry(company_name):
    industry_prompt = f"'{company_name}'의 산업군을 알려주세요."
    return chat(industry_prompt, []).strip()

# CSV 파일 로드 및 텍스트 분할 함수
def load_and_split_csv(file_path):
    df = pd.read_csv(file_path)
    text_data = df.to_string(index=False)  # 데이터프레임을 텍스트로 변환
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_text(text_data)
    return [split for split in splits]  # 리스트로 반환

# PDF 파일 로드 및 텍스트 분할 함수
def load_and_split_pdf(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    splits = text_splitter.split_documents(docs)
    return [doc.page_content for doc in splits]  # 리스트로 반환

# 문서 임베딩 및 FAISS 인덱싱
def embed_and_index_docs_with_faiss(splits, faiss_index_path="faiss_index"):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_texts(texts=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# FAISS 검색을 이용하여 관련 정보 추출 함수 (기업 분석용)
def retrieve_related_info_analysis(company_name):
    industry = extract_industry(company_name)
    combined_query = f"{company_name} {industry}"
    if db_retriever_analysis:
        search_results = db_retriever_analysis.get_relevant_documents(combined_query)
        return "\n".join([doc.page_content for doc in search_results])
    return "관련 정보를 찾을 수 없습니다."

# FAISS 검색을 이용하여 관련 정보 추출 함수 (매수/매도 추천용)
def retrieve_related_info_recommendation(company_name):
    industry = extract_industry(company_name)
    combined_query = f"{company_name} {industry}"
    if db_retriever_recommendation:
        search_results = db_retriever_recommendation.get_relevant_documents(combined_query)
        return "\n".join([doc.page_content for doc in search_results])
    return "관련 정보를 찾을 수 없습니다."

# 기업 분석 생성 함수
def generate_company_analysis(company_name):
    history = []
    related_info = retrieve_related_info_analysis(company_name)
    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수
def generate_trade_recommendation_with_analysis(company_name):
    history = []
    analysis = generate_company_analysis(company_name)
    related_info = retrieve_related_info_recommendation(company_name)
    recommendation_message = (
        f"다음은 '{company_name}'에 대한 기업 분석과 관련 정보입니다:\n\n{analysis}\n\n"
        f"이 정보를 바탕으로 매수 또는 매도 추천을 제공해 주세요:\n{related_info}"
    )
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 최종 매수/매도 결정 생성 함수
def generate_final_decision(analysis, recommendation):
    history = []
    final_decision_message = (
        f"다음은 기업 분석과 매수/매도 추천 내용입니다. "
        f"이 모든 정보를 종합하여 최종적으로 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요:\n\n"
        f"기업 분석:\n{analysis}\n\n매수/매도 추천:\n{recommendation}"
    )
    final_decision = chat(final_decision_message, history)
    if "매수" in final_decision:
        return "매수"
    elif "매도" in final_decision or "중립" in final_decision or "hold" in final_decision.lower():
        return "매도"
    else:
        return "추천 없음"

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("기업명을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="종목명", placeholder="예: 삼성전자")

    submit_btn = gr.Button("분석 및 추천 받기")
    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    submit_btn.click(
        lambda x: (generate_company_analysis(x), generate_trade_recommendation_with_analysis(x), generate_final_decision(generate_company_analysis(x), generate_trade_recommendation_with_analysis(x))),
        inputs=company_name,
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

if __name__ == "__main__":
    # CSV 데이터 임베딩
    test_splits = load_and_split_csv("test.csv")
    db_retriever_analysis = embed_and_index_docs_with_faiss(test_splits, "faiss_index_analysis")

    # PDF 데이터 임베딩
    hana_splits = load_and_split_pdf("hana.pdf")
    db_retriever_recommendation = embed_and_index_docs_with_faiss(hana_splits, "faiss_index_recommendation")

    demo.launch()

KeyboardInterrupt: 

In [40]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain.vectorstores import FAISS
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# CSV 파일 로딩 및 텍스트 변환
def load_and_convert_csv(file_path):
    df = pd.read_csv(file_path)
    text_data = "\n".join(df.apply(lambda row: " ".join(map(str, row)), axis=1))
    return text_data

# PDF 파일 로딩 및 텍스트 변환
def load_and_convert_pdf(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    return "\n".join([doc.page_content for doc in docs])

# FAISS 임베딩 생성 및 인덱스 저장
def embed_and_index_docs_with_faiss(text, faiss_index_path="faiss_index"):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    splits = text.split("\n")  # Split text into lines for FAISS indexing
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_texts(texts=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 기업명에 해당하는 데이터만 필터링하는 함수
def filter_company_data(text, company_name):
    pattern = rf"{company_name}.*?(?=(\n[A-Z가-힣]+|\Z))"
    matches = re.findall(pattern, text, re.DOTALL)
    return "\n".join(matches) if matches else f"{company_name}에 대한 데이터를 찾을 수 없습니다."

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name, retriever):
    industry = extract_industry(company_name)
    combined_query = f"{company_name} {industry}"
    
    if retriever:
        search_results = retriever.get_relevant_documents(combined_query)
        full_text = "\n".join([doc.page_content for doc in search_results])
        return filter_company_data(full_text, company_name)
    return "관련 정보를 찾을 수 없습니다."

# 산업군 추출 함수
def extract_industry(company_name):
    industry_prompt = f"'{company_name}'의 산업군을 알려주세요."
    return chat(industry_prompt, []).strip()

# 기업 분석 생성 함수
def generate_company_analysis(company_name, retriever):
    history = []
    related_info = retrieve_related_info(company_name, retriever)
    
    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수
def generate_trade_recommendation_with_analysis(company_name, retriever):
    history = []
    
    # 기업 분석과 관련 정보 결합
    analysis = generate_company_analysis(company_name, retriever)
    related_info = retrieve_related_info(company_name, retriever)

    recommendation_message = (
        f"다음은 '{company_name}'에 대한 기업 분석과 관련 정보입니다:\n\n{analysis}\n\n"
        f"이 정보를 바탕으로 반드시 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요. "
        f"단, 추천 이유와 함께 고려해야 할 추가 사항도 설명해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 최종 매수/매도 결정 생성 함수
def generate_final_decision(analysis, recommendation):
    history = []
    
    final_decision_message = (
        f"다음은 기업 분석과 매수/매도 추천 내용입니다. "
        f"이 모든 정보를 종합하여 최종적으로 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요:\n\n"
        f"기업 분석:\n{analysis}\n\n매수/매도 추천:\n{recommendation}"
    )
    final_decision = chat(final_decision_message, history)
    
    if "매수" in final_decision:
        return "매수"
    elif "매도" in final_decision or "중립" in final_decision or "hold" in final_decision.lower():
        return "매도"
    else:
        return "추천 없음"

# 통합 함수
def generate_analysis_recommendation_decision(company_name, retriever_analysis, retriever_recommendation):
    if not company_name:
        return "기업명을 입력해 주세요.", "", ""
    
    # 기업 분석 및 매수/매도 추천 생성
    analysis = generate_company_analysis(company_name, retriever_analysis)
    recommendation = generate_trade_recommendation_with_analysis(company_name, retriever_recommendation)
    
    # 최종 매수/매도 결론 생성
    final_decision = generate_final_decision(analysis, recommendation)
    
    return analysis, recommendation, final_decision

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("기업명을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="종목명", placeholder="예: 삼성전자")

    # 기업명 아래에 버튼 추가
    submit_btn = gr.Button("분석 및 추천 받기")

    # 기업 분석과 매수/매도 추천 및 최종 결정 결과 표시를 위한 텍스트 박스 컴포넌트 추가
    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    # 버튼 클릭 이벤트 설정 및 연결
    submit_btn.click(
        lambda company: generate_analysis_recommendation_decision(
            company, db_retriever_analysis, db_retriever_recommendation
        ),
        inputs=[company_name],
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

if __name__ == "__main__":
    # CSV 데이터 로드 및 임베딩 설정
    csv_text = load_and_convert_csv("test.csv")
    db_retriever_analysis = embed_and_index_docs_with_faiss(csv_text, "faiss_index_analysis")

    # PDF 데이터 로드 및 임베딩 설정
    pdf_text = load_and_convert_pdf("hana.pdf")
    db_retriever_recommendation = embed_and_index_docs_with_faiss(pdf_text, "faiss_index_recommendation")
    
    demo.launch()

* Running on local URL:  http://127.0.0.1:7926

To create a public link, set `share=True` in `launch()`.


In [15]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain.vectorstores import FAISS
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# CSV 파일 로딩 및 텍스트 변환
def load_and_convert_csv(file_path):
    df = pd.read_csv(file_path)
    text_data = "\n".join(df.apply(lambda row: " ".join(map(str, row)), axis=1))
    return text_data

# PDF 파일 로딩 및 텍스트 변환
def load_and_convert_pdf(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    return "\n".join([doc.page_content for doc in docs])

# FAISS 임베딩 생성 및 인덱스 저장
def embed_and_index_docs_with_faiss(text, faiss_index_path="faiss_index"):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    splits = text.split("\n")  # Split text into lines for FAISS indexing
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_texts(texts=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 기업명에 해당하는 데이터만 필터링하는 함수
def filter_company_data(text, company_name):
    pattern = rf"{company_name}.*?(?=(\n[A-Z가-힣]+|\Z))"
    matches = re.findall(pattern, text, re.DOTALL)
    return "\n".join(matches) if matches else f"{company_name}에 대한 데이터를 찾을 수 없습니다."

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name, retriever):
    industry = extract_industry(company_name)
    combined_query = f"{company_name} {industry}"
    
    print(f"검색 쿼리: {combined_query}")  # 검색 쿼리 출력

    if retriever:
        search_results = retriever.get_relevant_documents(combined_query)
        
        # 검색된 문서 내용 출력
        print("검색된 문서들:")
        for doc in search_results:
            print(doc.page_content)
        
        # 검색된 내용 중 해당 기업 데이터만 필터링
        full_text = "\n".join([doc.page_content for doc in search_results])
        filtered_data = filter_company_data(full_text, company_name)
        
        print(f"\n필터링된 '{company_name}' 관련 데이터:\n{filtered_data}")  # 필터링된 데이터 출력
        return filtered_data
    
    return "관련 정보를 찾을 수 없습니다."

# 산업군 추출 함수
def extract_industry(company_name):
    industry_prompt = f"'{company_name}'의 산업군을 알려주세요."
    return chat(industry_prompt, []).strip()

# 기업 분석 생성 함수
def generate_company_analysis(company_name, retriever):
    history = []
    related_info = retrieve_related_info(company_name, retriever)
    
    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수
def generate_trade_recommendation_with_analysis(company_name, retriever):
    history = []
    
    # 기업 분석과 관련 정보 결합
    analysis = generate_company_analysis(company_name, retriever)
    related_info = retrieve_related_info(company_name, retriever)

    recommendation_message = (
        f"다음은 '{company_name}'에 대한 기업 분석과 관련 정보입니다:\n\n{analysis}\n\n"
        f"이 정보를 바탕으로 반드시 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요. "
        f"단, 추천 이유와 함께 고려해야 할 추가 사항도 설명해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 최종 매수/매도 결정 생성 함수
def generate_final_decision(analysis, recommendation):
    history = []
    
    final_decision_message = (
        f"다음은 기업 분석과 매수/매도 추천 내용입니다. "
        f"이 모든 정보를 종합하여 최종적으로 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요:\n\n"
        f"기업 분석:\n{analysis}\n\n매수/매도 추천:\n{recommendation}"
    )
    final_decision = chat(final_decision_message, history)
    
    if "매수" in final_decision:
        return "매수"
    elif "매도" in final_decision or "중립" in final_decision or "hold" in final_decision.lower():
        return "매도"
    else:
        return "추천 없음"

# 통합 함수
def generate_analysis_recommendation_decision(company_name, retriever_analysis, retriever_recommendation):
    if not company_name:
        return "기업명을 입력해 주세요.", "", ""
    
    # 기업 분석 및 매수/매도 추천 생성
    analysis = generate_company_analysis(company_name, retriever_analysis)
    recommendation = generate_trade_recommendation_with_analysis(company_name, retriever_recommendation)
    
    # 최종 매수/매도 결론 생성
    final_decision = generate_final_decision(analysis, recommendation)
    
    return analysis, recommendation, final_decision

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("기업명을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="종목명", placeholder="예: 삼성전자")

    # 기업명 아래에 버튼 추가
    submit_btn = gr.Button("분석 및 추천 받기")

    # 기업 분석과 매수/매도 추천 및 최종 결정 결과 표시를 위한 텍스트 박스 컴포넌트 추가
    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    # 버튼 클릭 이벤트 설정 및 연결
    submit_btn.click(
        lambda company: generate_analysis_recommendation_decision(
            company, db_retriever_analysis, db_retriever_recommendation
        ),
        inputs=[company_name],
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

if __name__ == "__main__":
    # CSV 데이터 로드 및 임베딩 설정
    csv_text = load_and_convert_csv("test.csv")
    db_retriever_analysis = embed_and_index_docs_with_faiss(csv_text, "faiss_index_analysis")

    # PDF 데이터 로드 및 임베딩 설정
    pdf_text = load_and_convert_pdf("hana.pdf")
    db_retriever_recommendation = embed_and_index_docs_with_faiss(pdf_text, "faiss_index_recommendation")
    
    demo.launch()

* Running on local URL:  http://127.0.0.1:7905

To create a public link, set `share=True` in `launch()`.


검색 쿼리: 삼성전자 삼성전자는 다양한 산업군에 속해 있습니다. 주요 산업군은 다음과 같습니다:

1. 전자제품 제조: 스마트폰, TV, 가전제품, 컴퓨터, 카메라 등 다양한 전자제품을 생산합니다.

2. 반도체 제조: 메모리 반도체 (DRAM, NAND 플래시), 시스템 반도체, 파운드리 (위탁생산) 등 다양한 반도체 제품을 생산합니다.

3. 디스플레이 제조: 스마트폰 및 TV용 OLED (유기발광다이오드) 디스플레이, LCD (액정표시장치) 등을 생산합니다.

4. 통신 장비 제조: 이동통신 기지국, 통신 네트워크 장비 등을 생산합니다.

5. 의료 장비 제조: 의료용 영상 진단 장비, 의료용 IT 솔루션 등을 생산합니다.

6. 가전제품 제조: 냉장고, 세탁기, 에어컨, 청소기 등 다양한 가전제품을 생산합니다.

7. 조명 제조: LED 조명, 조명용 IT 솔루션 등을 생산합니다.

8. 금융 서비스: 삼성증권은 금융 서비스 산업군에 속하며, 주식, 채권, 펀드 등 다양한 금융 상품을 제공합니다.

9. 호텔 및 리조트 운영: 삼성은 호텔신라, 에버랜드 리조트 등 호텔 및 리조트 운영 사업도 진행하고 있습니다.

이는 일부이며, 삼성전자는 다양한 분야에서 사업을 진행하고 있습니다.
검색된 문서들:
Unnamed: 0     기업명                                                                                                                                                                                       개괄                                                                                                                             투자 의견  종목 코드
          0    삼성전자        삼성전자는

In [16]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain.vectorstores import FAISS
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# CSV 파일 로딩 및 텍스트 변환
def load_and_convert_csv(file_path):
    df = pd.read_csv(file_path)
    text_data = "\n".join(df.apply(lambda row: " ".join(map(str, row)), axis=1))
    return text_data

# PDF 파일 로딩 및 텍스트 변환
def load_and_convert_pdf(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    return "\n".join([doc.page_content for doc in docs])

# FAISS 임베딩 생성 및 인덱스 저장
def embed_and_index_docs_with_faiss(text, faiss_index_path="faiss_index"):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    splits = text.split("\n")  # Split text into lines for FAISS indexing
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_texts(texts=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 기업명에 해당하는 데이터만 필터링하는 함수
def filter_company_data(text, company_name):
    company_name_lower = company_name.lower()  # 대소문자 구분 없이 처리
    pattern = rf"{company_name_lower}.*?(?=(\n[A-Z가-힣]+|\Z))"
    matches = re.findall(pattern, text.lower(), re.DOTALL)  # 소문자로 통일하여 검색
    return "\n".join(matches) if matches else f"{company_name}에 대한 데이터를 찾을 수 없습니다."

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name, retriever):
    industry = extract_industry(company_name)
    combined_query = f"{company_name} {industry}".lower()  # 검색어를 소문자로 통일
    
    print(f"검색 쿼리: {combined_query}")  # 검색 쿼리 출력

    if retriever:
        search_results = retriever.get_relevant_documents(combined_query)
        
        # 검색된 문서 내용 출력
        print("검색된 문서들:")
        for doc in search_results:
            print(doc.page_content)
        
        # 검색된 내용 중 해당 기업 데이터만 필터링
        full_text = "\n".join([doc.page_content for doc in search_results])
        filtered_data = filter_company_data(full_text, company_name)
        
        print(f"\n필터링된 '{company_name}' 관련 데이터:\n{filtered_data}")  # 필터링된 데이터 출력
        return filtered_data
    
    return "관련 정보를 찾을 수 없습니다."

# 산업군 추출 함수
def extract_industry(company_name):
    industry_prompt = f"'{company_name}'의 산업군을 알려주세요."
    return chat(industry_prompt, []).strip()

# 기업 분석 생성 함수
def generate_company_analysis(company_name, retriever):
    history = []
    related_info = retrieve_related_info(company_name, retriever)
    
    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수
def generate_trade_recommendation_with_analysis(company_name, retriever):
    history = []
    
    # 기업 분석과 관련 정보 결합
    analysis = generate_company_analysis(company_name, retriever)
    related_info = retrieve_related_info(company_name, retriever)

    recommendation_message = (
        f"다음은 '{company_name}'에 대한 기업 분석과 관련 정보입니다:\n\n{analysis}\n\n"
        f"이 정보를 바탕으로 반드시 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요. "
        f"추가적인 고려 사항이 있을 경우 간략히 설명하고, 모호한 답변은 삼가 주세요. "
        f"분석에 기반하여 분명한 결정을 제공해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 최종 매수/매도 결정 생성 함수
def generate_final_decision(analysis, recommendation):
    history = []
    
    final_decision_message = (
        f"다음은 기업 분석과 매수/매도 추천 내용입니다. "
        f"이 모든 정보를 종합하여 최종적으로 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요:\n\n"
        f"기업 분석:\n{analysis}\n\n매수/매도 추천:\n{recommendation}"
    )
    final_decision = chat(final_decision_message, history)
    
    if "매수" in final_decision:
        return "매수"
    elif "매도" in final_decision or "중립" in final_decision or "hold" in final_decision.lower():
        return "매도"
    else:
        return "추천 없음"

# 통합 함수
def generate_analysis_recommendation_decision(company_name, retriever_analysis, retriever_recommendation):
    if not company_name:
        return "기업명을 입력해 주세요.", "", ""
    
    # 기업 분석 및 매수/매도 추천 생성
    analysis = generate_company_analysis(company_name, retriever_analysis)
    recommendation = generate_trade_recommendation_with_analysis(company_name, retriever_recommendation)
    
    # 최종 매수/매도 결론 생성
    final_decision = generate_final_decision(analysis, recommendation)
    
    return analysis, recommendation, final_decision

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("기업명을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="종목명", placeholder="예: 삼성전자")

    # 기업명 아래에 버튼 추가
    submit_btn = gr.Button("분석 및 추천 받기")

    # 기업 분석과 매수/매도 추천 및 최종 결정 결과 표시를 위한 텍스트 박스 컴포넌트 추가
    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    # 버튼 클릭 이벤트 설정 및 연결
    submit_btn.click(
        lambda company: generate_analysis_recommendation_decision(
            company, db_retriever_analysis, db_retriever_recommendation
        ),
        inputs=[company_name],
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

if __name__ == "__main__":
    # CSV 데이터 로드 및 임베딩 설정
    csv_text = load_and_convert_csv("test.csv")
    db_retriever_analysis = embed_and_index_docs_with_faiss(csv_text, "faiss_index_analysis")

    # PDF 데이터 로드 및 임베딩 설정
    pdf_text = load_and_convert_pdf("hana.pdf")
    db_retriever_recommendation = embed_and_index_docs_with_faiss(pdf_text, "faiss_index_recommendation")
    
    demo.launch()

* Running on local URL:  http://127.0.0.1:7906

To create a public link, set `share=True` in `launch()`.


검색 쿼리: sk하이닉스 sk하이닉스는 반도체 산업군에 속합니다.
검색된 문서들:
1  SK하이닉스                                SK하이닉스는 DRAM과 NAND 플래시 메모리에서 경쟁력을 갖춘 반도체 기업입니다. 3Q24 영업이익은 7.0조 원으로, HBM 및 고부가가치 메모리 제품의 수요 증가에 따라 실적이 개선되었습니다. 레거시 메모리 시장은 둔화되고 있으나, HBM의 강세가 지속되고 있습니다.                           목표주가는 300,000원으로 하향 조정되었지만, 여전히 매수를 추천합니다. 메모리 업계의 이익 체력이 높아지고 있으며, AI와 서버향 메모리 수요가 긍정적 요인으로 작용하고 있습니다.    660
          2   SK스퀘어                       SK스퀘어는 투자 및 지주사업을 영위하는 회사로, SK하이닉스와 같은 계열사의 실적에 큰 영향을 받습니다. 최근 SK하이닉스의 실적 개선이 SK스퀘어의 배당수입 증가로 이어질 가능성이 큽니다. 주요 자회사들의 매출 감소에도 불구하고, SK하이닉스의 실적 상향이 긍정적으로 작용하고 있습니다.                                                 매수를 유지하며 목표주가는 94,000원입니다. 주가가 할인된 상태로 평가되며, 향후 배당수입과 주주환원 정책이 투자 매력으로 부각되고 있습니다. 402340
Unnamed: 0     기업명                                                                                                                                                                                       개괄                                                                             

In [18]:
import os
import gradio as gr
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain.vectorstores import FAISS
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# CSV 파일 로딩 및 텍스트 변환
def load_and_convert_csv(file_path):
    df = pd.read_csv(file_path)
    text_data = "\n".join(df.apply(lambda row: " ".join(map(str, row)), axis=1))
    return text_data

# PDF 파일 로딩 및 텍스트 변환
def load_and_convert_pdf(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    return "\n".join([doc.page_content for doc in docs])

# FAISS 임베딩 생성 및 인덱스 저장
def embed_and_index_docs_with_faiss(text, faiss_index_path="faiss_index"):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    splits = text.split("\n")  # Split text into lines for FAISS indexing
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_texts(texts=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 기업명에 해당하는 데이터만 필터링하는 함수
def filter_company_data(text, company_name):
    company_name_lower = company_name.lower()  # 대소문자 구분 없이 처리
    pattern = rf"{company_name_lower}.*?(?=(\n[A-Z가-힣]+|\Z))"
    matches = re.findall(pattern, text.lower(), re.DOTALL)  # 소문자로 통일하여 검색
    return "\n".join(matches) if matches else f"{company_name}에 대한 데이터를 찾을 수 없습니다."

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name, retriever_list):
    industry = extract_industry(company_name)
    combined_query = f"{company_name} {industry}".lower()  # 검색어를 소문자로 통일
    
    print(f"검색 쿼리: {combined_query}")  # 검색 쿼리 출력

    # 모든 retriever에서 검색 수행
    full_text = ""
    for retriever in retriever_list:
        search_results = retriever.get_relevant_documents(combined_query)
        
        # 검색된 문서 내용 출력
        print("검색된 문서들:")
        for doc in search_results:
            print(doc.page_content)
        
        full_text += "\n".join([doc.page_content for doc in search_results])
    
    # 검색된 내용 중 해당 기업 데이터만 필터링
    filtered_data = filter_company_data(full_text, company_name)
    print(f"\n필터링된 '{company_name}' 관련 데이터:\n{filtered_data}")  # 필터링된 데이터 출력
    return filtered_data

# 산업군 추출 함수
def extract_industry(company_name):
    industry_prompt = f"'{company_name}'의 산업군을 알려주세요."
    return chat(industry_prompt, []).strip()

# 기업 분석 생성 함수
def generate_company_analysis(company_name, retriever_list):
    history = []
    related_info = retrieve_related_info(company_name, retriever_list)
    
    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수
def generate_trade_recommendation_with_analysis(company_name, retriever):
    history = []
    
    # 기업 분석과 관련 정보 결합
    analysis = generate_company_analysis(company_name, retriever)
    related_info = retrieve_related_info(company_name, retriever)

    recommendation_message = (
        f"다음은 '{company_name}'에 대한 기업 분석과 관련 정보입니다:\n\n{analysis}\n\n"
        f"이 정보를 바탕으로 반드시 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요. "
        f"추가적인 고려 사항이 있을 경우 간략히 설명하고, 모호한 답변은 삼가 주세요. "
        f"분명한 결정을 제공해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 최종 매수/매도 결정 생성 함수
def generate_final_decision(analysis, recommendation):
    history = []
    
    final_decision_message = (
        f"다음은 기업 분석과 매수/매도 추천 내용입니다. "
        f"이 모든 정보를 종합하여 최종적으로 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요:\n\n"
        f"기업 분석:\n{analysis}\n\n매수/매도 추천:\n{recommendation}"
    )
    final_decision = chat(final_decision_message, history)
    
    if "매수" in final_decision:
        return "매수"
    elif "매도" in final_decision or "중립" in final_decision or "hold" in final_decision.lower():
        return "매도"
    else:
        return "추천 없음"

# 통합 함수
def generate_analysis_recommendation_decision(company_name, retriever_analysis_list, retriever_recommendation):
    if not company_name:
        return "기업명을 입력해 주세요.", "", ""
    
    # 기업 분석 및 매수/매도 추천 생성
    analysis = generate_company_analysis(company_name, retriever_analysis_list)
    recommendation = generate_trade_recommendation_with_analysis(company_name, retriever_recommendation)
    
    # 최종 매수/매도 결론 생성
    final_decision = generate_final_decision(analysis, recommendation)
    
    return analysis, recommendation, final_decision

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("기업명을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="종목명", placeholder="예: 삼성전자")

    # 기업명 아래에 버튼 추가
    submit_btn = gr.Button("분석 및 추천 받기")

    # 기업 분석과 매수/매도 추천 및 최종 결정 결과 표시를 위한 텍스트 박스 컴포넌트 추가
    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    # 버튼 클릭 이벤트 설정 및 연결
    submit_btn.click(
        lambda company: generate_analysis_recommendation_decision(
            company, [db_retriever_test_csv, db_retriever_stock_csv], db_retriever_recommendation
        ),
        inputs=[company_name],
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

if __name__ == "__main__":
    # CSV 데이터 로드 및 임베딩 설정 (test.csv와 stock.csv 각각)
    test_csv_text = load_and_convert_csv("test.csv")
    db_retriever_test_csv = embed_and_index_docs_with_faiss(test_csv_text, "faiss_index_test_csv")
    
    stock_csv_text = load_and_convert_csv("stock.csv")
    db_retriever_stock_csv = embed_and_index_docs_with_faiss(stock_csv_text, "faiss_index_stock_csv")

    # PDF 데이터 로드 및 임베딩 설정
    pdf_text = load_and_convert_pdf("hana.pdf")
    db_retriever_recommendation = embed_and_index_docs_with_faiss(pdf_text, "faiss_index_recommendation")
    
    demo.launch()

* Running on local URL:  http://127.0.0.1:7908

To create a public link, set `share=True` in `launch()`.


In [24]:
import os
import gradio as gr
import sqlite3
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain.vectorstores import FAISS
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# CSV 파일 로딩 및 텍스트 변환
def load_and_convert_csv(file_path):
    df = pd.read_csv(file_path)
    text_data = "\n".join(df.apply(lambda row: " ".join(map(str, row)), axis=1))
    return text_data

# PDF 파일 로딩 및 텍스트 변환
def load_and_convert_pdf(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    return "\n".join([doc.page_content for doc in docs])

# SQLite 데이터베이스 로딩 및 텍스트 변환
def load_and_convert_db(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # 데이터베이스의 테이블 이름을 확인
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    if not tables:
        print("Warning: No tables found in the database.")
        conn.close()
        return ""
    
    # 모든 테이블의 데이터를 텍스트로 변환
    text_data = ""
    for table_name in tables:
        table_name = table_name[0]
        print(f"Processing table: {table_name}")  # 테이블 이름 출력
        
        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()
        
        if not rows:
            print(f"Warning: Table '{table_name}' is empty.")
        else:
            for row in rows:
                row_text = " ".join(map(str, row))
                text_data += row_text + "\n"
    
    conn.close()
    if not text_data.strip():
        print("Warning: No data found in any tables.")
    return text_data

# FAISS 임베딩 생성 및 인덱스 저장 (청크로 나누어 처리)
def embed_and_index_docs_with_faiss(text, faiss_index_path="faiss_index", chunk_size=500):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    splits = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]  # Chunk the text
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_texts(texts=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 기업명에 해당하는 데이터만 필터링하는 함수
def filter_company_data(text, company_name):
    company_name_lower = company_name.lower()  # 대소문자 구분 없이 처리
    pattern = rf"{company_name_lower}.*?(?=(\n[A-Z가-힣]+|\Z))"
    matches = re.findall(pattern, text.lower(), re.DOTALL)  # 소문자로 통일하여 검색
    return "\n".join(matches) if matches else f"{company_name}에 대한 데이터를 찾을 수 없습니다."

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name, retriever_list):
    industry = extract_industry(company_name)
    combined_query = f"{company_name} {industry}".lower()  # 검색어를 소문자로 통일
    
    print(f"검색 쿼리: {combined_query}")  # 검색 쿼리 출력

    # 모든 retriever에서 검색 수행
    full_text = ""
    for retriever in retriever_list:
        search_results = retriever.get_relevant_documents(combined_query)
        
        # 검색된 문서 내용 출력
        print("검색된 문서들:")
        for doc in search_results:
            print(doc.page_content)
        
        full_text += "\n".join([doc.page_content for doc in search_results])
    
    # 검색된 내용 중 해당 기업 데이터만 필터링
    filtered_data = filter_company_data(full_text, company_name)
    print(f"\n필터링된 '{company_name}' 관련 데이터:\n{filtered_data}")  # 필터링된 데이터 출력
    return filtered_data

# 산업군 추출 함수
def extract_industry(company_name):
    industry_prompt = f"'{company_name}'의 산업군을 알려주세요."
    return chat(industry_prompt, []).strip()

# 기업 분석 생성 함수
def generate_company_analysis(company_name, retriever_list):
    history = []
    related_info = retrieve_related_info(company_name, retriever_list)
    
    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수
def generate_trade_recommendation_with_analysis(company_name, retriever):
    history = []
    
    # 기업 분석과 관련 정보 결합
    analysis = generate_company_analysis(company_name, retriever)
    related_info = retrieve_related_info(company_name, retriever)

    recommendation_message = (
        f"다음은 '{company_name}'에 대한 기업 분석과 관련 정보입니다:\n\n{analysis}\n\n"
        f"이 정보를 바탕으로 반드시 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요. "
        f"추가적인 고려 사항이 있을 경우 간략히 설명하고, 모호한 답변은 삼가 주세요. "
        f"분명한 결정을 제공해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 최종 매수/매도 결정 생성 함수
def generate_final_decision(analysis, recommendation):
    history = []
    
    final_decision_message = (
        f"다음은 기업 분석과 매수/매도 추천 내용입니다. "
        f"이 모든 정보를 종합하여 최종적으로 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요:\n\n"
        f"기업 분석:\n{analysis}\n\n매수/매도 추천:\n{recommendation}"
    )
    final_decision = chat(final_decision_message, history)
    
    if "매수" in final_decision:
        return "매수"
    elif "매도" in final_decision or "중립" in final_decision or "hold" in final_decision.lower():
        return "매도"
    else:
        return "추천 없음"

# 통합 함수
def generate_analysis_recommendation_decision(company_name, retriever_analysis_list, retriever_recommendation):
    if not company_name:
        return "기업명을 입력해 주세요.", "", ""
    
    # 기업 분석 및 매수/매도 추천 생성
    analysis = generate_company_analysis(company_name, retriever_analysis_list)
    recommendation = generate_trade_recommendation_with_analysis(company_name, retriever_recommendation)
    
    # 최종 매수/매도 결론 생성
    final_decision = generate_final_decision(analysis, recommendation)
    
    return analysis, recommendation, final_decision

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("기업명을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="종목명", placeholder="예: 삼성전자")

    # 기업명 아래에 버튼 추가
    submit_btn = gr.Button("분석 및 추천 받기")

    # 기업 분석과 매수/매도 추천 및 최종 결정 결과 표시를 위한 텍스트 박스 컴포넌트 추가
    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    # 버튼 클릭 이벤트 설정 및 연결
    submit_btn.click(
        lambda company: generate_analysis_recommendation_decision(
            company, [db_retriever_test_csv, db_retriever_stock_csv, db_retriever_info_stock], db_retriever_recommendation
        ),
        inputs=[company_name],
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

# FAISS 임베딩 생성 및 인덱스 저장 (청크로 나누어 처리)
def embed_and_index_docs_with_faiss(text, faiss_index_path="faiss_index", chunk_size=500):
    if not text.strip():  # 텍스트가 비어 있으면 함수 종료
        print(f"Warning: The text for {faiss_index_path} is empty. Skipping embedding.")
        return None
    
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    splits = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]  # 청크로 분할

    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_texts(texts=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 기존 코드와 결합
if __name__ == "__main__":
    # CSV 데이터 로드 및 임베딩 설정 (test.csv와 stock.csv 각각)
    test_csv_text = load_and_convert_csv("test.csv")
    db_retriever_test_csv = embed_and_index_docs_with_faiss(test_csv_text, "faiss_index_test_csv")
    
    stock_csv_text = load_and_convert_csv("stock.csv")
    db_retriever_stock_csv = embed_and_index_docs_with_faiss(stock_csv_text, "faiss_index_stock_csv")

    # DB 데이터 로드 및 임베딩 설정 (info_stock.db)
    info_stock_text = load_and_convert_db("info_stock.db")
    db_retriever_info_stock = embed_and_index_docs_with_faiss(info_stock_text, "faiss_index_info_stock")

    # PDF 데이터 로드 및 임베딩 설정
    pdf_text = load_and_convert_pdf("hana.pdf")
    db_retriever_recommendation = embed_and_index_docs_with_faiss(pdf_text, "faiss_index_recommendation")
    
    demo.launch()

Processing table: stock_data
Processing table: stock_info
* Running on local URL:  http://127.0.0.1:7910

To create a public link, set `share=True` in `launch()`.


In [25]:
import os
import gradio as gr
import sqlite3
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain.vectorstores import FAISS
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# CSV 파일 로딩 및 텍스트 변환
def load_and_convert_csv(file_path):
    df = pd.read_csv(file_path)
    text_data = "\n".join(df.apply(lambda row: " ".join(map(str, row)), axis=1))
    return text_data

# PDF 파일 로딩 및 텍스트 변환
def load_and_convert_pdf(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    return "\n".join([doc.page_content for doc in docs])

# SQLite 데이터베이스에서 특정 종목명에 해당하는 데이터 추출
def load_and_extract_from_db(db_path, company_name):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # 특정 종목명에 해당하는 데이터만 추출
    cursor.execute(f"SELECT * FROM stock_info WHERE 종목명 = ?", (company_name,))
    rows = cursor.fetchall()
    
    conn.close()
    
    # 데이터가 없을 경우 경고 메시지 반환
    if not rows:
        print(f"Warning: '{company_name}'에 해당하는 데이터를 찾을 수 없습니다.")
        return f"{company_name}에 대한 데이터를 찾을 수 없습니다."
    
    # 데이터를 텍스트 형태로 변환
    text_data = "\n".join(" ".join(map(str, row)) for row in rows)
    print(f"추출된 '{company_name}' 데이터:\n{text_data}")
    return text_data

# FAISS 임베딩 생성 및 인덱스 저장 (청크로 나누어 처리)
def embed_and_index_docs_with_faiss(text, faiss_index_path="faiss_index", chunk_size=500):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    splits = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]  # Chunk the text
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_texts(texts=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 기업명에 해당하는 데이터만 필터링하는 함수
def filter_company_data(text, company_name):
    company_name_lower = company_name.lower()  # 대소문자 구분 없이 처리
    pattern = rf"{company_name_lower}.*?(?=(\n[A-Z가-힣]+|\Z))"
    matches = re.findall(pattern, text.lower(), re.DOTALL)  # 소문자로 통일하여 검색
    return "\n".join(matches) if matches else f"{company_name}에 대한 데이터를 찾을 수 없습니다."

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name, retriever_list, db_path):
    industry = extract_industry(company_name)
    combined_query = f"{company_name} {industry}".lower()  # 검색어를 소문자로 통일
    
    print(f"검색 쿼리: {combined_query}")  # 검색 쿼리 출력

    # 모든 retriever에서 검색 수행
    full_text = ""
    for retriever in retriever_list:
        search_results = retriever.get_relevant_documents(combined_query)
        
        # 검색된 문서 내용 출력
        print("검색된 문서들:")
        for doc in search_results:
            print(doc.page_content)
        
        full_text += "\n".join([doc.page_content for doc in search_results])
    
    # info_stock.db에서 해당 기업 데이터 추출
    db_text = load_and_extract_from_db(db_path, company_name)
    full_text += "\n" + db_text
    
    # 검색된 내용 중 해당 기업 데이터만 필터링
    filtered_data = filter_company_data(full_text, company_name)
    print(f"\n필터링된 '{company_name}' 관련 데이터:\n{filtered_data}")  # 필터링된 데이터 출력
    return filtered_data

# 산업군 추출 함수
def extract_industry(company_name):
    industry_prompt = f"'{company_name}'의 산업군을 알려주세요."
    return chat(industry_prompt, []).strip()

# 기업 분석 생성 함수
def generate_company_analysis(company_name, retriever_list, db_path):
    history = []
    related_info = retrieve_related_info(company_name, retriever_list, db_path)
    
    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수
def generate_trade_recommendation_with_analysis(company_name, retriever, db_path):
    history = []
    
    # 기업 분석과 관련 정보 결합
    analysis = generate_company_analysis(company_name, retriever, db_path)
    related_info = retrieve_related_info(company_name, retriever, db_path)

    recommendation_message = (
        f"다음은 '{company_name}'에 대한 기업 분석과 관련 정보입니다:\n\n{analysis}\n\n"
        f"이 정보를 바탕으로 반드시 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요. "
        f"추가적인 고려 사항이 있을 경우 간략히 설명하고, 모호한 답변은 삼가 주세요. "
        f"분명한 결정을 제공해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 최종 매수/매도 결정 생성 함수
def generate_final_decision(analysis, recommendation):
    history = []
    
    final_decision_message = (
        f"다음은 기업 분석과 매수/매도 추천 내용입니다. "
        f"이 모든 정보를 종합하여 최종적으로 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요:\n\n"
        f"기업 분석:\n{analysis}\n\n매수/매도 추천:\n{recommendation}"
    )
    final_decision = chat(final_decision_message, history)
    
    if "매수" in final_decision:
        return "매수"
    elif "매도" in final_decision or "중립" in final_decision or "hold" in final_decision.lower():
        return "매도"
    else:
        return "추천 없음"

# 통합 함수
def generate_analysis_recommendation_decision(company_name, retriever_analysis_list, retriever_recommendation, db_path):
    if not company_name:
        return "기업명을 입력해 주세요.", "", ""
    
    # 기업 분석 및 매수/매도 추천 생성
    analysis = generate_company_analysis(company_name, retriever_analysis_list, db_path)
    recommendation = generate_trade_recommendation_with_analysis(company_name, retriever_recommendation, db_path)
    
    # 최종 매수/매도 결론 생성
    final_decision = generate_final_decision(analysis, recommendation)
    
    return analysis, recommendation, final_decision

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("기업명을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="종목명", placeholder="예: 삼성전자")

    # 기업명 아래에 버튼 추가
    submit_btn = gr.Button("분석 및 추천 받기")

    # 기업 분석과 매수/매도 추천 및 최종 결정 결과 표시를 위한 텍스트 박스 컴포넌트 추가
    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    # 버튼 클릭 이벤트 설정 및 연결
    submit_btn.click(
        lambda company: generate_analysis_recommendation_decision(
            company, [db_retriever_test_csv, db_retriever_stock_csv], db_retriever_recommendation, "info_stock.db"
        ),
        inputs=[company_name],
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

# FAISS 임베딩 생성 및 인덱스 저장 (청크로 나누어 처리)
def embed_and_index_docs_with_faiss(text, faiss_index_path="faiss_index", chunk_size=500):
    if not text.strip():  # 텍스트가 비어 있으면 함수 종료
        print(f"Warning: The text for {faiss_index_path} is empty. Skipping embedding.")
        return None
    
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    splits = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]  # 청크로 분할

    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_texts(texts=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 기존 코드와 결합
if __name__ == "__main__":
    # CSV 데이터 로드 및 임베딩 설정 (test.csv와 stock.csv 각각)
    test_csv_text = load_and_convert_csv("test.csv")
    db_retriever_test_csv = embed_and_index_docs_with_faiss(test_csv_text, "faiss_index_test_csv")
    
    stock_csv_text = load_and_convert_csv("stock.csv")
    db_retriever_stock_csv = embed_and_index_docs_with_faiss(stock_csv_text, "faiss_index_stock_csv")

    # PDF 데이터 로드 및 임베딩 설정
    pdf_text = load_and_convert_pdf("hana.pdf")
    db_retriever_recommendation = embed_and_index_docs_with_faiss(pdf_text, "faiss_index_recommendation")
    
    demo.launch()

* Running on local URL:  http://127.0.0.1:7911

To create a public link, set `share=True` in `launch()`.


검색 쿼리: 삼성전자 삼성전자는 다양한 산업 분야에 걸쳐 사업을 진행하고 있습니다. 주요 산업군은 다음과 같습니다:

1. 전자제품: 스마트폰, 태블릿, 웨어러블 기기, 가전제품, 텔레비전, 컴퓨터, 프린터, 디지털 카메라 등 다양한 소비자 전자제품을 생산합니다.
2. 반도체: 메모리 반도체 (dram, nand flash), 시스템 반도체 (logic, foundry), ssd (solid state drive) 등 다양한 반도체 제품을 제조합니다.
3. 디스플레이: lcd (액정 디스플레이), oled (유기 발광 다이오드), qd-oled (퀀텀닷 oled) 등 다양한 디스플레이 패널을 제조합니다.
4. 통신 네트워크: 5g, 4g, 3g 등의 이동통신 네트워크 장비 및 솔루션을 제공합니다.
5. 의료 기기: 의료 영상 진단 장비, 의료용 디스플레이, 의료 정보 시스템 등 다양한 의료 기기를 생산합니다.
6. 디지털 헬스: 디지털 헬스케어 솔루션 및 서비스를 개발하여 개인 건강 관리에 기여합니다.
7. 사물인터넷 (iot): 스마트 홈, 스마트 빌딩, 스마트 시티 등의 분야에서 iot 제품과 솔루션을 제공합니다.

이는 일부이며, 삼성전자는 지속적으로 혁신과 다각화를 추구하며 새로운 산업 분야에 진출하고 있습니다.
검색된 문서들:
0 삼성전자 삼성전자는 글로벌 반도체 및 전자기기 제조업체로, 메모리 반도체와 파운드리 사업에서 큰 비중을 차지합니다. 3Q24 실적은 영업이익 9.18조 원으로 시장 예상치를 하회했으며, HBM3E와 서버향 메모리 수요 증대에 집중하고 있습니다. 주요 이슈는 DRAM 및 NAND 수익성 방어와 비메모리 부문의 실적 부진입니다. 투자의견은 대부분 '매수'로 유지되고 있으며, 목표주가는 76,000~87,000원으로 설정되었습니다. 실적 개선은 2025년 예상되며, HBM 공급 확대 및 고부가가치 제품 위주의 전략이 향후 주가에 긍정적 영향을 미칠 것입니다. 5930
1 SK하이닉스 SK하이닉스는 DRAM과 NAND 플래시 메모리

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/blocks.py", line 2018, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/gradio/blocks.py", line 1567, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/L

In [26]:
import os
import gradio as gr
import sqlite3
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain.vectorstores import FAISS
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# CSV 파일 로딩 및 텍스트 변환
def load_and_convert_csv(file_path):
    df = pd.read_csv(file_path)
    text_data = "\n".join(df.apply(lambda row: " ".join(map(str, row)), axis=1))
    return text_data

# PDF 파일 로딩 및 텍스트 변환
def load_and_convert_pdf(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    return "\n".join([doc.page_content for doc in docs])

# SQLite 데이터베이스에서 특정 종목명에 해당하는 데이터 추출
def load_and_extract_from_db(db_path, company_name):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute(f"SELECT * FROM stock_info WHERE 종목명 = ?", (company_name,))
    rows = cursor.fetchall()
    conn.close()
    
    if not rows:
        print(f"Warning: '{company_name}'에 해당하는 데이터를 찾을 수 없습니다.")
        return f"{company_name}에 대한 데이터를 찾을 수 없습니다."
    
    text_data = "\n".join(" ".join(map(str, row)) for row in rows)
    print(f"추출된 '{company_name}' 데이터:\n{text_data}")
    return text_data

# FAISS 임베딩 생성 및 인덱스 저장 (청크로 나누어 처리)
def embed_and_index_docs_with_faiss(text, faiss_index_path="faiss_index", chunk_size=500):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    splits = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_texts(texts=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 기업명에 해당하는 데이터만 필터링하는 함수
def filter_company_data(text, company_name):
    company_name_lower = company_name.lower()
    pattern = rf"{company_name_lower}.*?(?=(\n[A-Z가-힣]+|\Z))"
    matches = re.findall(pattern, text.lower(), re.DOTALL)
    return "\n".join(matches) if matches else f"{company_name}에 대한 데이터를 찾을 수 없습니다."

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name, retriever_list, db_path):
    industry = extract_industry(company_name)
    combined_query = f"{company_name} {industry}".lower()
    
    print(f"검색 쿼리: {combined_query}")

    full_text = ""
    for retriever in retriever_list:
        search_results = retriever.get_relevant_documents(combined_query)
        print("검색된 문서들:")
        for doc in search_results:
            print(doc.page_content)
        
        full_text += "\n".join([doc.page_content for doc in search_results])
    
    db_text = load_and_extract_from_db(db_path, company_name)
    full_text += "\n" + db_text
    
    filtered_data = filter_company_data(full_text, company_name)
    print(f"\n필터링된 '{company_name}' 관련 데이터:\n{filtered_data}")
    return filtered_data

# 산업군 추출 함수
def extract_industry(company_name):
    industry_prompt = f"'{company_name}'의 산업군을 알려주세요."
    return chat(industry_prompt, []).strip()

# 기업 분석 생성 함수
def generate_company_analysis(company_name, retriever_list, db_path):
    history = []
    related_info = retrieve_related_info(company_name, retriever_list, db_path)
    
    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수
def generate_trade_recommendation_with_analysis(company_name, retriever_list, db_path):
    history = []
    analysis = generate_company_analysis(company_name, retriever_list, db_path)
    related_info = retrieve_related_info(company_name, retriever_list, db_path)

    recommendation_message = (
        f"다음은 '{company_name}'에 대한 기업 분석과 관련 정보입니다:\n\n{analysis}\n\n"
        f"이 정보를 바탕으로 반드시 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요. "
        f"추가적인 고려 사항이 있을 경우 간략히 설명하고, 모호한 답변은 삼가 주세요. "
        f"분명한 결정을 제공해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 최종 매수/매도 결정 생성 함수
def generate_final_decision(analysis, recommendation):
    history = []
    
    final_decision_message = (
        f"다음은 기업 분석과 매수/매도 추천 내용입니다. "
        f"이 모든 정보를 종합하여 최종적으로 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요:\n\n"
        f"기업 분석:\n{analysis}\n\n매수/매도 추천:\n{recommendation}"
    )
    final_decision = chat(final_decision_message, history)
    
    if "매수" in final_decision:
        return "매수"
    elif "매도" in final_decision or "중립" in final_decision or "hold" in final_decision.lower():
        return "매도"
    else:
        return "추천 없음"

# 통합 함수
def generate_analysis_recommendation_decision(company_name, retriever_analysis_list, db_path):
    if not company_name:
        return "기업명을 입력해 주세요.", "", ""
    
    analysis = generate_company_analysis(company_name, retriever_analysis_list, db_path)
    recommendation = generate_trade_recommendation_with_analysis(company_name, retriever_analysis_list, db_path)
    final_decision = generate_final_decision(analysis, recommendation)
    
    return analysis, recommendation, final_decision

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("기업명을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="종목명", placeholder="예: 삼성전자")

    submit_btn = gr.Button("분석 및 추천 받기")

    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    submit_btn.click(
        lambda company: generate_analysis_recommendation_decision(
            company, [db_retriever_test_csv, db_retriever_stock_csv], "info_stock.db"
        ),
        inputs=[company_name],
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

# 기존 코드와 결합
if __name__ == "__main__":
    test_csv_text = load_and_convert_csv("test.csv")
    db_retriever_test_csv = embed_and_index_docs_with_faiss(test_csv_text, "faiss_index_test_csv")
    
    stock_csv_text = load_and_convert_csv("stock.csv")
    db_retriever_stock_csv = embed_and_index_docs_with_faiss(stock_csv_text, "faiss_index_stock_csv")

    pdf_text = load_and_convert_pdf("hana.pdf")
    db_retriever_recommendation = embed_and_index_docs_with_faiss(pdf_text, "faiss_index_recommendation")
    
    demo.launch()

* Running on local URL:  http://127.0.0.1:7912

To create a public link, set `share=True` in `launch()`.


검색 쿼리: 삼성전자 삼성전자는 다양한 산업군을 아우르는 다국적 기업이지만, 주로 전자 제품 제조 및 판매에 집중하고 있습니다. 주요 산업군은 다음과 같습니다:

1. 가전제품: tv, 냉장고, 세탁기, 에어컨, 청소기 등
2. 스마트폰: 갤럭시 시리즈 등
3. 컴퓨터 및 주변기기: 노트북, 데스크탑, 모니터, 프린터 등
4. 반도체: 메모리 칩, 시스템 lsi, led 등
5. 디스플레이: lcd, oled 등
6. 의료 기기: 의료 영상 기기, 디지털 x-ray 등
7. 통신 시스템: 네트워크 장비, lte, 5g 등

이 외에도 삼성전자는 다양한 분야에서 사업을 진행하고 있으며, 기술 혁신과 제품 다양화를 통해 글로벌 시장에서 선도적인 위치를 차지하고 있습니다.
검색된 문서들:
0 삼성전자 삼성전자는 글로벌 반도체 및 전자기기 제조업체로, 메모리 반도체와 파운드리 사업에서 큰 비중을 차지합니다. 3Q24 실적은 영업이익 9.18조 원으로 시장 예상치를 하회했으며, HBM3E와 서버향 메모리 수요 증대에 집중하고 있습니다. 주요 이슈는 DRAM 및 NAND 수익성 방어와 비메모리 부문의 실적 부진입니다. 투자의견은 대부분 '매수'로 유지되고 있으며, 목표주가는 76,000~87,000원으로 설정되었습니다. 실적 개선은 2025년 예상되며, HBM 공급 확대 및 고부가가치 제품 위주의 전략이 향후 주가에 긍정적 영향을 미칠 것입니다. 5930
1 SK하이닉스 SK하이닉스는 DRAM과 NAND 플래시 메모리에서 경쟁력을 갖춘 반도체 기업입니다. 3Q24 영업이익은 7.0조 원으로, HBM 및 고부가가치 메모리 제품의 수요 증가에 따라 실적이 개선되었습니다. 레거시 메모리 시장은 둔화되고 있으나, HBM의 강세가 지속되고 있습니다. 목표주가는 300,000원으로 하향 조정되었지만, 여전히 매수를 추천합니다. 메모리 업계의 이익 체력이 높아지고 있으며, AI와 서버향 메모리 수요가 긍정적 요인으로 작용하고 있습니다. 660
8 테크윙 테크윙은 반도체 검사 장비 전

In [27]:
import os
import gradio as gr
import sqlite3
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain.vectorstores import FAISS
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# CSV 파일 로딩 및 텍스트 변환
def load_and_convert_csv(file_path):
    df = pd.read_csv(file_path)
    text_data = "\n".join(df.apply(lambda row: " ".join(map(str, row)), axis=1))
    return text_data

# PDF 파일 로딩 및 텍스트 변환
def load_and_convert_pdf(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    return "\n".join([doc.page_content for doc in docs])

# stock.csv에서 종목명을 기반으로 종목 코드를 찾는 함수
def get_stock_code_from_csv(stock_csv_path, company_name):
    df = pd.read_csv(stock_csv_path)
    row = df[df['종목명'] == company_name]
    
    if row.empty:
        print(f"Warning: '{company_name}'에 해당하는 종목코드를 찾을 수 없습니다.")
        return None
    return row['종목코드'].values[0]

# SQLite 데이터베이스에서 특정 종목코드에 해당하는 데이터 추출
def load_and_extract_from_db(db_path, company_name, stock_csv_path):
    stock_code = get_stock_code_from_csv(stock_csv_path, company_name)
    if stock_code is None:
        return f"{company_name}에 대한 종목코드를 찾을 수 없습니다."
    
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute("SELECT * FROM stock_info WHERE 종목코드 = ?", (stock_code,))
    rows = cursor.fetchall()
    conn.close()
    
    if not rows:
        print(f"Warning: '{company_name}'에 해당하는 데이터를 찾을 수 없습니다.")
        return f"{company_name}에 대한 데이터를 찾을 수 없습니다."
    
    text_data = "\n".join(" ".join(map(str, row)) for row in rows)
    print(f"추출된 '{company_name}' 데이터:\n{text_data}")
    return text_data

# FAISS 임베딩 생성 및 인덱스 저장 (청크로 나누어 처리)
def embed_and_index_docs_with_faiss(text, faiss_index_path="faiss_index", chunk_size=500):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    splits = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_texts(texts=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 기업명에 해당하는 데이터만 필터링하는 함수
def filter_company_data(text, company_name):
    company_name_lower = company_name.lower()
    pattern = rf"{company_name_lower}.*?(?=(\n[A-Z가-힣]+|\Z))"
    matches = re.findall(pattern, text.lower(), re.DOTALL)
    return "\n".join(matches) if matches else f"{company_name}에 대한 데이터를 찾을 수 없습니다."

# FAISS 검색을 이용하여 관련 정보 추출 함수
def retrieve_related_info(company_name, retriever_list, db_path, stock_csv_path):
    industry = extract_industry(company_name)
    combined_query = f"{company_name} {industry}".lower()
    
    print(f"검색 쿼리: {combined_query}")

    full_text = ""
    for retriever in retriever_list:
        search_results = retriever.get_relevant_documents(combined_query)
        print("검색된 문서들:")
        for doc in search_results:
            print(doc.page_content)
        
        full_text += "\n".join([doc.page_content for doc in search_results])
    
    db_text = load_and_extract_from_db(db_path, company_name, stock_csv_path)
    full_text += "\n" + db_text
    
    filtered_data = filter_company_data(full_text, company_name)
    print(f"\n필터링된 '{company_name}' 관련 데이터:\n{filtered_data}")
    return filtered_data

# 산업군 추출 함수
def extract_industry(company_name):
    industry_prompt = f"'{company_name}'의 산업군을 알려주세요."
    return chat(industry_prompt, []).strip()

# 기업 분석 생성 함수
def generate_company_analysis(company_name, retriever_list, db_path, stock_csv_path):
    history = []
    related_info = retrieve_related_info(company_name, retriever_list, db_path, stock_csv_path)
    
    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수
def generate_trade_recommendation_with_analysis(company_name, retriever_list, db_path, stock_csv_path):
    history = []
    analysis = generate_company_analysis(company_name, retriever_list, db_path, stock_csv_path)
    related_info = retrieve_related_info(company_name, retriever_list, db_path, stock_csv_path)

    recommendation_message = (
        f"다음은 '{company_name}'에 대한 기업 분석과 관련 정보입니다:\n\n{analysis}\n\n"
        f"이 정보를 바탕으로 반드시 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요. "
        f"추가적인 고려 사항이 있을 경우 간략히 설명하고, 모호한 답변은 삼가 주세요. "
        f"분명한 결정을 제공해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 최종 매수/매도 결정 생성 함수
def generate_final_decision(analysis, recommendation):
    history = []
    
    final_decision_message = (
        f"다음은 기업 분석과 매수/매도 추천 내용입니다. "
        f"이 모든 정보를 종합하여 최종적으로 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요:\n\n"
        f"기업 분석:\n{analysis}\n\n매수/매도 추천:\n{recommendation}"
    )
    final_decision = chat(final_decision_message, history)
    
    if "매수" in final_decision:
        return "매수"
    elif "매도" in final_decision or "중립" in final_decision or "hold" in final_decision.lower():
        return "매도"
    else:
        return "추천 없음"

# 통합 함수
def generate_analysis_recommendation_decision(company_name, retriever_analysis_list, db_path, stock_csv_path):
    if not company_name:
        return "기업명을 입력해 주세요.", "", ""
    
    analysis = generate_company_analysis(company_name, retriever_analysis_list, db_path, stock_csv_path)
    recommendation = generate_trade_recommendation_with_analysis(company_name, retriever_analysis_list, db_path, stock_csv_path)
    final_decision = generate_final_decision(analysis, recommendation)
    
    return analysis, recommendation, final_decision

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("기업명을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="종목명", placeholder="예: 삼성전자")

    submit_btn = gr.Button("분석 및 추천 받기")

    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    submit_btn.click(
        lambda company: generate_analysis_recommendation_decision(
            company, [db_retriever_test_csv, db_retriever_stock_csv], "info_stock.db", "stock.csv"
        ),
        inputs=[company_name],
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

if __name__ == "__main__":
    test_csv_text = load_and_convert_csv("test.csv")
    db_retriever_test_csv = embed_and_index_docs_with_faiss(test_csv_text, "faiss_index_test_csv")
    
    stock_csv_text = load_and_convert_csv("stock.csv")
    db_retriever_stock_csv = embed_and_index_docs_with_faiss(stock_csv_text, "faiss_index_stock_csv")
    
    pdf_text = load_and_convert_pdf("hana.pdf")
    db_retriever_recommendation = embed_and_index_docs_with_faiss(pdf_text, "faiss_index_recommendation")
    
    demo.launch()

* Running on local URL:  http://127.0.0.1:7913

To create a public link, set `share=True` in `launch()`.


검색 쿼리: 삼성전자 삼성전자는 다양한 산업군에 속해 있지만, 주로 전자 제품 및 정보통신 기술(ict) 산업에 속해 있습니다. 주요 사업 부문은 다음과 같습니다:

1. 디바이스 경험(device experience): 스마트폰, 태블릿, 웨어러블 기기, 네트워크 시스템 등을 개발하고 제조합니다.
2. 디지털 미디어 & 가전(digital media & appliances): tv, 홈 어플라이언스, 에어컨, 청소기 등의 가전 제품을 생산합니다.
3. it & 모바일 커뮤니케이션(it & mobile communications): 컴퓨터, 프린터, 네트워크 시스템, 스마트폰, 웨어러블 기기 등을 개발하고 제조합니다.
4. 반도체(semiconductor): 메모리 반도체, 시스템 반도체, 파운드리(foundry) 서비스를 제공합니다.
5. 디스플레이(display): lcd, oled, qd-oled 등의 디스플레이 패널을 생산합니다.

이 외에도 삼성전자는 다양한 분야에서 사업을 진행하고 있으며, 예를 들어 의료 장비, 조명, 자동차 전장 부품 등도 포함됩니다.
검색된 문서들:
0 삼성전자 삼성전자는 글로벌 반도체 및 전자기기 제조업체로, 메모리 반도체와 파운드리 사업에서 큰 비중을 차지합니다. 3Q24 실적은 영업이익 9.18조 원으로 시장 예상치를 하회했으며, HBM3E와 서버향 메모리 수요 증대에 집중하고 있습니다. 주요 이슈는 DRAM 및 NAND 수익성 방어와 비메모리 부문의 실적 부진입니다. 투자의견은 대부분 '매수'로 유지되고 있으며, 목표주가는 76,000~87,000원으로 설정되었습니다. 실적 개선은 2025년 예상되며, HBM 공급 확대 및 고부가가치 제품 위주의 전략이 향후 주가에 긍정적 영향을 미칠 것입니다. 5930
4 DB하이텍 DB하이텍은 8인치 웨이퍼 파운드리 업체로, 가전 수요 회복 기대에 힘입어 점진적인 실적 개선을 전망하고 있습니다. 2024년 매출은 1.1조 원, 영업이익은 1,920억 원을 예상하며, ASP 하락세가 완

In [36]:
import os
import gradio as gr
import sqlite3
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain.vectorstores import FAISS
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# CSV 파일 로딩 및 텍스트 변환
def load_and_convert_csv(file_path):
    df = pd.read_csv(file_path)
    text_data = "\n".join(df.apply(lambda row: " ".join(map(str, row)), axis=1))
    return text_data

# PDF 파일 로딩 및 텍스트 변환
def load_and_convert_pdf(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    return "\n".join([doc.page_content for doc in docs])

# SQLite 데이터베이스에서 특정 종목명에 해당하는 데이터 추출 (대소문자 구분 없이)
def load_and_extract_from_db(db_path, company_name):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    # 대소문자 구분 없이 검색
    cursor.execute("SELECT * FROM stock_info WHERE LOWER(종목명) = ?", (company_name.lower(),))
    rows = cursor.fetchall()
    conn.close()
    
    if not rows:
        print(f"Warning: '{company_name}'에 해당하는 데이터를 찾을 수 없습니다.")
        return f"종목명 '{company_name}'에 대한 데이터를 찾을 수 없습니다."
    
    text_data = "\n".join(" ".join(map(str, row)) for row in rows)
    print(f"추출된 종목명 '{company_name}' 데이터:\n{text_data}")
    return text_data

# FAISS 임베딩 생성 및 인덱스 저장 (청크로 나누어 처리)
def embed_and_index_docs_with_faiss(text, faiss_index_path="faiss_index", chunk_size=500):
    embeddings = UpstageEmbeddings(model="solar-embedding-1-large")
    splits = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    
    if os.path.exists(faiss_index_path):
        vectorstore = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
    else:
        vectorstore = FAISS.from_texts(texts=splits, embedding=embeddings)
        vectorstore.save_local(faiss_index_path)
    
    return vectorstore.as_retriever(search_kwargs={"k": 3})

# 기업명에 해당하는 데이터만 필터링하는 함수
def filter_company_data(text, company_name):
    company_name_lower = company_name.lower()
    pattern = rf"{company_name_lower}.*?(?=(\n[A-Z가-힣]+|\Z))"
    matches = re.findall(pattern, text.lower(), re.DOTALL)
    return "\n".join(matches) if matches else f"{company_name}에 대한 데이터를 찾을 수 없습니다."

# FAISS 검색을 이용하여 관련 정보 추출 함수 (산업군 없이 종목명만 사용)
def retrieve_related_info(company_name, retriever_list, db_path):
    print(f"검색 종목명: {company_name}")

    full_text = ""
    for retriever in retriever_list:
        search_results = retriever.get_relevant_documents(company_name.lower())
        print("검색된 문서들:")
        for doc in search_results:
            print(doc.page_content)
        
        full_text += "\n".join([doc.page_content for doc in search_results])
    
    # db에서 종목명만 사용하여 검색
    db_text = load_and_extract_from_db(db_path, company_name)
    full_text += "\n" + db_text
    
    filtered_data = filter_company_data(full_text, company_name)
    print(f"\n필터링된 '{company_name}' 관련 데이터:\n{filtered_data}")
    return filtered_data

# 산업군 추출 함수
def extract_industry(company_name):
    industry_prompt = f"'{company_name}'의 산업군을 알려주세요."
    return chat(industry_prompt, []).strip()

# 기업 분석 생성 함수
def generate_company_analysis(company_name, db_path):
    history = []
    related_info = load_and_extract_from_db(db_path, company_name)
    
    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{related_info}"
    )
    analysis = chat(analysis_message, history)
    return analysis.strip()

# 매수/매도 추천 생성 함수
def generate_trade_recommendation_with_analysis(company_name, retriever_list, db_path):
    history = []
    analysis = generate_company_analysis(company_name, db_path)
    related_info = retrieve_related_info(company_name, retriever_list, db_path)

    recommendation_message = (
        f"다음은 '{company_name}'에 대한 기업 분석과 관련 정보입니다:\n\n{analysis}\n\n"
        f"이 정보를 바탕으로 반드시 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요. "
        f"추가적인 고려 사항이 있을 경우 간략히 설명하고, 모호한 답변은 삼가 주세요. "
        f"분명한 결정을 제공해 주세요:\n{related_info}"
    )
    
    recommendation = chat(recommendation_message, history)
    return recommendation.strip()

# 최종 매수/매도 결정 생성 함수
def generate_final_decision(analysis, recommendation):
    history = []
    
    final_decision_message = (
        f"다음은 기업 분석과 매수/매도 추천 내용입니다. "
        f"이 모든 정보를 종합하여 최종적으로 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요:\n\n"
        f"기업 분석:\n{analysis}\n\n매수/매도 추천:\n{recommendation}"
    )
    final_decision = chat(final_decision_message, history)
    
    if "매수" in final_decision:
        return "매수"
    elif "매도" in final_decision or "중립" in final_decision or "hold" in final_decision.lower():
        return "매도"
    else:
        return "추천 없음"

# 통합 함수 수정
def generate_analysis_recommendation_decision(company_name, retriever_list, db_path):
    if not company_name:
        return "기업명을 입력해 주세요.", "", ""
    
    analysis = generate_company_analysis(company_name, db_path)
    recommendation = generate_trade_recommendation_with_analysis(company_name, retriever_list, db_path)
    final_decision = generate_final_decision(analysis, recommendation)
    
    return analysis, recommendation, final_decision

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("기업명을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="종목명", placeholder="예: 삼성전자")

    submit_btn = gr.Button("분석 및 추천 받기")

    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")
    final_decision_result = gr.Textbox(label="최종 결정", lines=2, placeholder="최종 매수/매도 결정을 여기에 표시합니다.")

    # FAISS 검색 객체 생성
    test_csv_text = load_and_convert_csv("test.csv")
    db_retriever_test_csv = embed_and_index_docs_with_faiss(test_csv_text, "faiss_index_test_csv")
    
    stock_csv_text = load_and_convert_csv("stock.csv")
    db_retriever_stock_csv = embed_and_index_docs_with_faiss(stock_csv_text, "faiss_index_stock_csv")
    
    pdf_text = load_and_convert_pdf("hana.pdf")
    db_retriever_recommendation = embed_and_index_docs_with_faiss(pdf_text, "faiss_index_recommendation")

    # retriever_list로 검색 객체 리스트 전달
    submit_btn.click(
        lambda company: generate_analysis_recommendation_decision(company, [db_retriever_test_csv, db_retriever_stock_csv, db_retriever_recommendation], "info_stock.db"),
        inputs=[company_name],
        outputs=[company_analysis_result, trade_recommendation_result, final_decision_result]
    )

    demo.launch()



* Running on local URL:  http://127.0.0.1:7923

To create a public link, set `share=True` in `launch()`.


검색 종목명: 삼성전자
검색된 문서들:
0 삼성전자 삼성전자는 글로벌 반도체 및 전자기기 제조업체로, 메모리 반도체와 파운드리 사업에서 큰 비중을 차지합니다. 3Q24 실적은 영업이익 9.18조 원으로 시장 예상치를 하회했으며, HBM3E와 서버향 메모리 수요 증대에 집중하고 있습니다. 주요 이슈는 DRAM 및 NAND 수익성 방어와 비메모리 부문의 실적 부진입니다. 투자의견은 대부분 '매수'로 유지되고 있으며, 목표주가는 76,000~87,000원으로 설정되었습니다. 실적 개선은 2025년 예상되며, HBM 공급 확대 및 고부가가치 제품 위주의 전략이 향후 주가에 긍정적 영향을 미칠 것입니다. 5930
5 솔브레인 솔브레인은 반도체 소재 전문 업체로, 인산계와 불산계 화학재료에서 독보적인 점유율을 보유하고 있습니다. 기술적 반등이 기대되는 시점이며, 메모리 가동률 회복에 따른 실적 개선 가능성이 큽니다. 주요 소재는 DRAM/NAND 및 3D NAND 질화막 식각에 사용됩니다. 투자의견은 '매수'로 신규 편입되었으며, 목표주가는 350,000원입니다. 2025년에는 300단대 NAND 시장 진입과 테크 마이그레이션에 따른 소재 수요 증가로 긍정적 흐름이 기대됩니다. 357780
4 DB하이텍 DB하이텍은 8인치 웨이퍼 파운드리 업체로, 가전 수요 회복 기대에 힘입어 점진적인 실적 개선을 전망하고 있습니다. 2024년 매출은 1.1조 원, 영업이익은 1,920억 원을 예상하며, ASP 하락세가 완화되고 가동률이 상승세를 보일 것으로 보고 있습니다. GaN과 SiC 소재 반도체의 장기적 성장 가능성도 주목받고 있습니다. 투자의견은 '매수'로 상향되었으며, 목표주가는 54,000원입니다. 향후 신제품 출시와 수요 회복이 기대되며, 가동률 상승과 소재 시장의 성장으로 실적 개선이 예상됩니다. 990
검색된 문서들:
삼성전자 5930 58300 59000.0 58300.0 -1.5202702702702704 348038322665000
솔브레인 357780 19620

In [37]:
import os
import gradio as gr
import sqlite3
from langchain_upstage import (
    ChatUpstage,
    UpstageEmbeddings,
    UpstageDocumentParseLoader,
)
from langchain.vectorstores import FAISS
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 한글 폰트 설정
def set_korean_font():
    if fm.findSystemFonts(fontpaths=None, fontext='ttf'):
        plt.rc('font', family='NanumGothic')
    else:
        print("한글 폰트를 찾을 수 없습니다. 영어 레이블을 사용합니다.")

set_korean_font()

# LLM 인스턴스 생성
def create_llm():
    return ChatUpstage()

# chat 함수 정의
def chat(message, history):
    llm = create_llm()
    response = llm(message)
    return response.content if hasattr(response, 'content') else str(response)

# SQLite 데이터베이스에서 특정 종목명에 해당하는 데이터 추출
def load_and_extract_from_db(db_path, company_name):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    
    cursor.execute(f"SELECT * FROM stock_info WHERE 종목명 = ?", (company_name,))
    rows = cursor.fetchall()
    conn.close()
    
    if not rows:
        print(f"Warning: '{company_name}'에 해당하는 데이터를 찾을 수 없습니다.")
        return f"{company_name}에 대한 데이터를 찾을 수 없습니다."
    
    text_data = "\n".join(" ".join(map(str, row)) for row in rows)
    print(f"추출된 '{company_name}' 데이터:\n{text_data}")
    return text_data

# PDF 파일 로딩 및 텍스트 변환
def load_and_convert_pdf(file_path):
    doc_loader = UpstageDocumentParseLoader(file_path, output_format='html')
    docs = doc_loader.load()
    return "\n".join([doc.page_content for doc in docs])

# 기업 분석 생성 함수
def generate_company_analysis(company_name, db_path):
    company_data = load_and_extract_from_db(db_path, company_name)
    
    if "데이터를 찾을 수 없습니다." in company_data:
        return company_data
    
    analysis_message = (
        f"'{company_name}'에 대한 기업 분석을 제공해 주세요. "
        f"다음 관련 정보도 포함해 분석을 제공해 주세요:\n{company_data}"
    )
    analysis = chat(analysis_message, [])
    return analysis.strip()

# 매수/매도 추천 생성 함수
def generate_trade_recommendation(company_name, db_path, pdf_retriever):
    company_analysis = generate_company_analysis(company_name, db_path)
    
    if "데이터를 찾을 수 없습니다." in company_analysis:
        return company_analysis
    
    search_results = pdf_retriever.get_relevant_documents(company_name)
    pdf_content = "\n".join([doc.page_content for doc in search_results])
    
    recommendation_message = (
        f"다음은 '{company_name}'에 대한 기업 분석과 PDF에서 추출한 관련 정보입니다:\n\n{company_analysis}\n\n"
        f"{pdf_content}\n\n이 정보를 바탕으로 반드시 '매수' 또는 '매도' 중 하나를 선택하여 추천해 주세요."
    )
    
    recommendation = chat(recommendation_message, [])
    return recommendation.strip()

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("# 관심 종목 매수/매도 추천")
    gr.Markdown("기업명을 입력하면 해당 기업의 분석과 매수/매도 추천을 제공합니다.")
    
    with gr.Row():
        company_name = gr.Textbox(label="종목명", placeholder="예: 삼성전자")

    submit_btn = gr.Button("분석 및 추천 받기")

    company_analysis_result = gr.Textbox(label="기업 분석", lines=5, placeholder="여기에 기업 분석이 표시됩니다.")
    trade_recommendation_result = gr.Textbox(label="매수/매도 추천", lines=5, placeholder="여기에 매수/매도 추천이 표시됩니다.")

    submit_btn.click(
        lambda company: (
            generate_company_analysis(company, "info_stock.db"),
            generate_trade_recommendation(company, "info_stock.db", db_retriever_recommendation),
        ),
        inputs=[company_name],
        outputs=[company_analysis_result, trade_recommendation_result]
    )

# 기존 코드와 결합
if __name__ == "__main__":
    pdf_text = load_and_convert_pdf("hana.pdf")
    db_retriever_recommendation = embed_and_index_docs_with_faiss(pdf_text, "faiss_index_recommendation")
    
    demo.launch()

* Running on local URL:  http://127.0.0.1:7924

To create a public link, set `share=True` in `launch()`.
